In [26]:
# Setup autoreload
%load_ext autoreload
%autoreload 2

# Give yourself access to common
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
# Gather training and testing data and process it to be configured for time intervals
from common import *
from sklearn.model_selection import train_test_split
import pandas as pd

nba = get_next_year_external_data()

FUT_SAL_CLASS = 'future_salary_class'
max = nba[NEXT_Y_SAL].max()
min = nba[NEXT_Y_SAL].min()
nba[FUT_SAL_CLASS] = nba[NEXT_Y_SAL].apply(lambda x: get_salary_class(x,max,min))
# feats = get_extern_features()
feats = get_base_features()
feats.append(FUT_SAL_CLASS)

X = pd.DataFrame(columns=feats)
y = pd.DataFrame()

for i, row in nba.iterrows() :
    next_val = nba.loc[((nba[P_NAME] == row[P_NAME]) & (nba[SZN_START_Y] == (row[SZN_START_Y]+1)))]  
    if not next_val.empty :
        nba.loc[i,'y'] = next_val.index[0]
    else :
        nba.loc[i,'y'] = np.nan

X = nba.copy(deep=True)
X = X.dropna()
y = nba.loc[X['y']]

In [28]:
# Normalize the data
# print(X.columns)
# print(y.columns)
X = X[feats]
y = y[feats]
print(X.columns)
print(y.columns)
# Scale the stuff down
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
y_scaled = scaler.transform(y)
y_scaled = y_scaled[:,:(len(get_base_features()))]

# Sanity check the scaler
print((X['FG']).iloc[0])
print(X_scaled[0, X.columns.get_loc('FG')])
inv = scaler.inverse_transform(X_scaled)
print(inv[0, X.columns.get_loc('FG')])


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size = 0.25, random_state=42)

Index(['MP', 'PTS', 'Age', 'games', 'games_started', 'PER', 'FTr', 'AST',
       'STL', 'TRB', 'FT', '3P', 'FG', 'height', 'weight', 'seasonStartYear',
       'startYear', 'all_star_total', 'all_star_enc', 'all_nba_enc',
       'all_nba_total', 'draft_pick', 'champion', 'conference_champ', 'mvp',
       'mvp_rank', 'mvp_total', 'player_week_enc', 'player_week_total', 'dpoy',
       'dpoy_rank', 'dpoy_total', 'future_salary_class'],
      dtype='object')
Index(['MP', 'PTS', 'Age', 'games', 'games_started', 'PER', 'FTr', 'AST',
       'STL', 'TRB', 'FT', '3P', 'FG', 'height', 'weight', 'seasonStartYear',
       'startYear', 'all_star_total', 'all_star_enc', 'all_nba_enc',
       'all_nba_total', 'draft_pick', 'champion', 'conference_champ', 'mvp',
       'mvp_rank', 'mvp_total', 'player_week_enc', 'player_week_total', 'dpoy',
       'dpoy_rank', 'dpoy_total', 'future_salary_class'],
      dtype='object')
655.0
2.228594445658938
655.0


In [29]:

# Fit the model
from sklearn.neural_network import MLPRegressor
import numpy as np

# MLPClassifier only classifies data as integers or strings, therefore, our problem is one of regression for the neural network
# Consequently I should use the mlp regressor
mlp = MLPRegressor(hidden_layer_sizes=(10,10,10), solver='adam', max_iter=1000)
mlp.fit(X_train,y_train)

# Evaluate the model
import numpy as np
from sklearn.metrics import mean_squared_error
predict_test = mlp.predict(X_test)
test_set_rsquared = mlp.score(X_test, y_test)
test_set_rmse = np.sqrt(mean_squared_error(predict_test, y_test))
test_set_mse = mean_squared_error(predict_test, y_test)
print('R_squared value: ', test_set_rsquared) # Variances are not nicely correlated
print('RMSE: ', test_set_rmse) # Fairly good at data prediction
print('MSE:', test_set_mse)
y_test

R_squared value:  0.5985229671951953
RMSE:  0.6551491530065454
MSE: 0.42922041268519384


array([[ 0.8784,  0.5664,  1.503 , ...,  0.5788,  1.6408,  0.7872],
       [ 0.8065,  0.8959,  1.2436, ...,  0.9473,  0.8153,  0.1395],
       [ 0.3332,  0.2244,  0.7249, ...,  0.3237, -0.5605,  0.1036],
       ...,
       [ 0.0577, -0.0595, -0.3124, ..., -0.0505, -0.5605, -0.2203],
       [ 0.6994,  0.8628, -0.5718, ...,  0.9643, -1.3859, -0.4721],
       [ 0.4734,  0.2058,  0.7249, ...,  0.1423, -0.0101, -0.2203]])

In [30]:
# Denormalize the data
predict_test = mlp.predict((X_scaled[0]).reshape(1,-1))
print(X_train.shape[1])
print(predict_test)
tmp = np.zeros((1,X_train.shape[1]))
tmp[:,:predict_test.shape[1]] = predict_test
print(tmp)
undone = scaler.inverse_transform(tmp)
print(undone.flatten().tolist()[:13])
print(y[get_base_features()].iloc[0])

33
[[ 0.0837  0.6395  1.9885 -0.2563  0.1779  1.2658  1.4456 -0.5428 -0.6458
   0.8913  1.0369 -0.663   0.7304  1.7721  1.0699]]
[[ 0.0837  0.6395  1.9885 -0.2563  0.1779  1.2658  1.4456 -0.5428 -0.6458
   0.8913  1.0369 -0.663   0.7304  1.7721  1.0699  0.      0.      0.
   0.      0.      0.      0.      0.      0.      0.      0.      0.
   0.      0.      0.      0.      0.      0.    ]]
[1745.0678853833579, 1009.2770367390337, 33.87184330438497, 58.43818875641746, 42.42270482521228, 20.392981318796444, 0.5584433678393097, 72.60930494638221, 30.146916627467295, 477.7716626631144, 257.02906644125653, 7.460894534901946, 390.7389309020293]
MP               848.000
PTS              540.000
Age               35.000
games             26.000
games_started     26.000
PER               23.300
FTr                0.462
AST               28.000
STL               16.000
TRB              265.000
FT               134.000
3P                 0.000
FG               203.000
height            84.000
w

In [31]:
# Compute the confidence interval of each of the features
from common import *

# Get the unscaled predictions
predict_full = mlp.predict(X_scaled)
tmp = np.zeros(X_scaled.shape)
tmp[:,:predict_full.shape[1]] = predict_full
predicted_unscaled_vals = scaler.inverse_transform(tmp)
predicted_unscaled_vals = predicted_unscaled_vals[:,:predict_full.shape[1]]

# compute the RMSE for each feature in the output vector
actual_unscaled_vals = y[get_base_features()].values
rmse = np.sqrt(np.square(predicted_unscaled_vals - actual_unscaled_vals))

# compute the percentilse of the RMSE to get the confidence interval of each predicted feature
ci_up = np.percentile(rmse, 95, axis=0) # 95th percentile
ci_low = np.percentile(rmse, 5, axis=0) # 5th percentile
print("Off by at most this much:", ci_up.flatten().tolist())
print(np.max(actual_unscaled_vals, axis=0))
print(np.min(actual_unscaled_vals, axis=0)) 
# print(ci_low.flatten().tolist()) 

# TODO: validate these findings with the very basic network to see if it predicts that these stats will land us in our desired class
np.set_printoptions(suppress=True, precision=4)
print(confidence_interval_numpy(predicted_unscaled_vals, actual_unscaled_vals)) # requires the assumption of gaussianity, but seems to be working?


print(bootstrap_confidence_interval(predicted_unscaled_vals, actual_unscaled_vals)) # should be able to work without the assumption of gaussianity
# computing for rmse, so says that the true root mean squared error has a 95% of landing in these intervals
# residuals are likely far smaller because you have the negatives balancing things out so it is going to be less accurate for computing the confidence interval

residuals = predicted_unscaled_vals - actual_unscaled_vals
ci = np.quantile(residuals, 1 - 0.05, axis=0)
print("HELLO")
print(ci)

# I want to say that my prediction is between these 2 bounds with 95% confidence
# The two functions I provided seem to find the rmse of each parameter with 95% confidence
# the first and 4th method I try do not scale the mean of the data and therefore, I don't think they necessarily apply to giving me the confidence interval I desire



# DO A CASE STUDY on an individual
# ANALYZE THIS MODEL ON BASELINE AND EXTERNAL




Off by at most this much: [1163.130727511755, 569.8916775464158, 1.3083402996734628, 35.7985248979969, 43.924468356065454, 5.5748947578512675, 0.19573023049432778, 177.40656491928468, 50.065130631436, 278.6203507102703, 134.81834674112804, 65.63120625328656, 215.47248126445828, 1.5506634277808182, 11.809877308196148]
[3485.  2832.    40.    85.    83.    45.3    6.   925.   225.  1226.
  756.   402.   978.    90.   325. ]
[  1.    0.   19.    1.    0.  -40.7   0.    0.    0.    0.    0.    0.
   0.   63.  135. ]
[[477.89   498.4262]
 [215.9425 226.0847]
 [  0.4079   0.4386]
 [ 13.69    14.3045]
 [ 18.4693  19.2184]
 [  2.1044   2.2227]
 [  0.0704   0.0766]
 [ 56.9093  60.2209]
 [ 18.2957  19.1819]
 [ 96.9404 101.991 ]
 [ 48.4484  50.9743]
 [ 20.5899  21.8274]
 [ 82.5575  86.3891]
 [  0.5393   0.5711]
 [  4.2366   4.4635]]
{'column_0': (477.305878795477, 497.4138690145247), 'column_1': (216.3853769696766, 226.07537867142577), 'column_2': (0.4082456708604416, 0.43996051105639045), 'colum

In [32]:
# from sklearn.ensemble import RandomForestRegressor 
# random_forest_model = RandomForestRegressor(n_estimators = 60, random_state = 0)

# random_forest_model.fit(X_train, y_train) 

# y_pred = random_forest_model.predict(X_test)

# mse = mean_squared_error(y_test, y_pred)
# rmse = np.sqrt(mean_squared_error(y_test, y_pred))
# print("MSE: ",mse)
# print("RMSE: ",rmse)
